#**Install Libraries**

In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 3.4 MB/s eta 0:00:00


In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-ps850gw7
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-ps850gw7
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=ff6d80a8408fb3d897409162c3eaa6371d15f7e5dc6f584441f8c1e68565f6e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-0i8bgbig/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


#**Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import LSTM, Embedding, Dense, Bidirectional, Input, TimeDistributed
from keras_contrib.layers import CRF
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow_addons.layers import CRF

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

#**Load Dataset**

In [ ]:
df = pd.read_csv('Final_200.csv')
df

,Text,Catchphrases
0,"Dipak Misra, J.\n1. Leave granted.\n2. The App...",Leave granted\nFIR No. 645 of 2004\nCharge she...
1,"U.U. Lalit, J.\n1. State Bank of India and 13 ...",Special leave petitions\nHigh Court of Karnata...
2,"﻿A.M. Khanwilkar, J. \n1. Leave granted. \nJUD...",Lease Deed\nDemand notice\nShow cause notice\n...
3,"Ranjan Gogoi, J.\n1. Leave granted.\n2 . The w...",Writ petition\nAppeals\nAgartala Bench of the ...
4,"Jasti Chelameswar, J.\n1. Leave Granted\n2 . M...","Negotiable Instruments Act, 1881\nSections 138..."
...,...,...
195,"﻿N.V. Ramana, J. \n1. These appeals by special...",Suit\nDeclaration of title\nPermanent injuncti...
196,"﻿Madan B. Lokur, J.\n1. Leave granted.\nJUDGME...",Section 92 of the Code of Civil Procedure: Thi...
197,"﻿Abhay Manohar Sapre, J. 1. Leave granted. \n2...",Order 6 Rule 1 the institution of the suit.\nO...
198,"﻿Aniruddha Bose, J. \n1. Leave granted. \nJUDG...","Marine Cargo-Open Policy, Sales Turnover basis..."


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Text          200 non-null    object
 1   Catchphrases  200 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB
None


#**Pre-processing**

In [ ]:
df['Text'] = df['Text'].apply(lambda x: x.lower())
df['Text'] = df['Text'].str.replace('[^\w\s]', '')

stop_words = set(stopwords.words('english'))
df['Text'] = df['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [ ]:
df['Text']

0      dipak misra, j. 1. leave granted. 2. appellant...
1      u.u. lalit, j. 1. state bank india 13 banks1 f...
2      ﻿a.m. khanwilkar, j. 1. leave granted. judgmen...
3      ranjan gogoi, j. 1. leave granted. 2 . writ pe...
4      jasti chelameswar, j. 1. leave granted 2 . m/s...
                             ...                        
195    ﻿n.v. ramana, j. 1. appeals special leave file...
196    ﻿madan b. lokur, j. 1. leave granted. judgment...
197    ﻿abhay manohar sapre, j. 1. leave granted. 2 ....
198    ﻿aniruddha bose, j. 1. leave granted. judgment...
199    ﻿b.r. gavai, j. 1. appeals challenge common ju...
Name: Text, Length: 200, dtype: object

#**Word Embedding**

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Text'])
vocab_size = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(df['Text'])
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [ ]:
labels = []

for i, text in enumerate(df['Text']):
    text_words = text.split()
    catchphrase_words = df['Catchphrases'][i].split()
    label_seq = []
    for word in text_words:
        if word in catchphrase_words:
            label_seq.append(0)  # 'B'
            catchphrase_words.remove(word)
        elif label_seq and label_seq[-1] == 0:
            label_seq.append(1)  # 'I'
        else:
            label_seq.append(2)  # 'O'

    label_seq += [2] * (max_length - len(label_seq))
    labels.append(label_seq)

labels = np.array(labels)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['Text'])]
doc2vec_model = Doc2Vec(documents, vector_size=100, window=3, min_count=1, workers=4)

#**Embedding Matrix**

In [ ]:
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = doc2vec_model.wv[word]
    except KeyError:
        embedding_vector = np.random.uniform(-0.25, 0.25, embedding_dim)
    embedding_matrix[i] = embedding_vector

In [ ]:
print(embedding_vector)

[ 0.03146301 -0.04241385 -0.01739126  0.06113508  0.00776357 -0.1940488
 -0.24638997  0.10171869  0.01145234 -0.05911075 -0.16306732  0.02559243
  0.15414191 -0.12232738 -0.10890217 -0.03966722 -0.07090547  0.08275879
 -0.00872382  0.04897892  0.12934095 -0.03939585  0.11620134  0.16144918
  0.08651817  0.06143934  0.2206835   0.01321031  0.19566756  0.14011854
 -0.23253964  0.15094848 -0.09806223 -0.07587779 -0.08708489 -0.13376489
  0.02552228 -0.17036379  0.21111715 -0.10593357 -0.17827871 -0.16598403
  0.03052925 -0.09660225 -0.00133673  0.18377165 -0.08152463  0.06473488
  0.16028268 -0.01392791  0.13466776  0.01760714  0.2213569  -0.10896495
 -0.22873621  0.17036638 -0.12718521 -0.08618734 -0.00949721 -0.23034158
  0.03713617 -0.12075659 -0.01283773 -0.05809856 -0.06023437 -0.23429089
 -0.22627569 -0.08333825  0.00116785 -0.16472417 -0.08127919  0.23044871
 -0.15774182  0.07368158  0.18924578  0.06060995  0.22432711 -0.21642997
  0.04706064 -0.0096659  -0.20385028 -0.19126999  0.

In [ ]:
print(embedding_matrix)

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.19834372 -0.21657379  0.24375728 ... -0.11446495  0.1138432
  -0.12549954]
 [ 0.19733731 -0.16246201  0.15119903 ...  0.11992944  0.0555561
  -0.1015736 ]
 ...
 [-0.24250117  0.000805   -0.16631529 ... -0.24461556 -0.05271532
   0.09498772]
 [ 0.1372467   0.07440186  0.01323015 ...  0.19475185 -0.02124131
   0.14908556]
 [ 0.03146301 -0.04241385 -0.01739126 ... -0.04404167  0.05034039
  -0.22862651]]


#**Bi-LSTM Model**

In [ ]:
inputs = Input(shape=(max_length,))

embedding = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)(inputs)
bilstm = Bidirectional(LSTM(256, return_sequences=True))(embedding)
bilstm1 = Bidirectional(LSTM(128, return_sequences=True))(bilstm)
bilstm2 = Bidirectional(LSTM(64, return_sequences=True))(bilstm1)
bilstm3 = Bidirectional(LSTM(32, return_sequences=True))(bilstm2)
output = TimeDistributed(Dense(3, activation='softmax'))(bilstm3)

model = Model(inputs, output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3484)]            0         
                                                                 
 embedding (Embedding)       (None, 3484, 100)         1441800   
                                                                 
 bidirectional (Bidirection  (None, 3484, 512)         731136    
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 3484, 256)         656384    
 onal)                                                           
                                                                 
 bidirectional_2 (Bidirecti  (None, 3484, 128)         164352    
 onal)                                                           
                                                             

#**Model Train**

In [ ]:
model.fit(padded_sequences, labels, epochs=5, batch_size=32)

Epoch 1/5
7/7 [==============================] - 30s 998ms/step - loss: 0.6346 - accuracy: 0.8350
Epoch 2/5
7/7 [==============================] - 6s 851ms/step - loss: 0.2010 - accuracy: 0.9587
Epoch 3/5
7/7 [==============================] - 6s 816ms/step - loss: 0.2007 - accuracy: 0.9587
Epoch 4/5
7/7 [==============================] - 6s 870ms/step - loss: 0.2010 - accuracy: 0.9587
Epoch 5/5
7/7 [==============================] - 6s 817ms/step - loss: 0.2004 - accuracy: 0.9587


In [ ]:
loss, accuracy = model.evaluate(padded_sequences, labels)
print("Loss: %.2f" % (loss*100))
print("Accuracy: %.2f" % (accuracy*100))

7/7 [==============================] - 6s 374ms/step - loss: 0.2002 - accuracy: 0.9587
Loss: 20.02
Accuracy: 95.87


#**Install Libraries for CRF**

In [ ]:
pip install sklearn-crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install python-crfsuite

#**Import Libraries for CRF**

In [ ]:
import keras_contrib
from keras_contrib.layers import CRF
from sklearn_crfsuite import CRF
import sklearn_crfsuite
import pycrfsuite
from sklearn_crfsuite import metrics

#**CRF Model**

In [ ]:
crf = CRF(algorithm='lbfgs', c1=1.0, c2=0.001)
crf_output = output

In [ ]:
from keras.utils import to_categorical

one_hot_labels = to_categorical(labels, num_classes=3)

model = Model(inputs=inputs, outputs=crf_output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(padded_sequences, one_hot_labels, epochs=5, batch_size=32, validation_data=(padded_sequences, one_hot_labels))


Epoch 1/5
7/7 [==============================] - 39s 3s/step - loss: 0.1247 - accuracy: 0.9587 - val_loss: 0.1252 - val_accuracy: 0.9587
Epoch 2/5
7/7 [==============================] - 8s 1s/step - loss: 0.1244 - accuracy: 0.9587 - val_loss: 0.1247 - val_accuracy: 0.9587
Epoch 3/5
7/7 [==============================] - 9s 1s/step - loss: 0.1245 - accuracy: 0.9587 - val_loss: 0.1247 - val_accuracy: 0.9587
Epoch 4/5
7/7 [==============================] - 8s 1s/step - loss: 0.1244 - accuracy: 0.9587 - val_loss: 0.1243 - val_accuracy: 0.9587
Epoch 5/5
7/7 [==============================] - 9s 1s/step - loss: 0.1244 - accuracy: 0.9587 - val_loss: 0.1243 - val_accuracy: 0.9587


#**Predictions**

In [ ]:
predictions = model.predict(padded_sequences)
print(predictions)

7/7 [==============================] - 5s 389ms/step
[[[0.08243462 0.07389201 0.84367335]
  [0.06009332 0.04416538 0.8957413 ]
  [0.04801075 0.03065212 0.9213371 ]
  ...
  [0.03010805 0.02063832 0.94925356]
  [0.03574431 0.02440508 0.9398507 ]
  [0.05533995 0.03740406 0.907256  ]]

 [[0.06783316 0.061314   0.87085277]
  [0.04372946 0.03476739 0.9215031 ]
  [0.03326113 0.02482508 0.9419138 ]
  ...
  [0.03010804 0.02063832 0.94925356]
  [0.03574431 0.02440508 0.9398507 ]
  [0.05533995 0.03740406 0.907256  ]]

 [[0.08031955 0.07178751 0.8478929 ]
  [0.05765367 0.0425005  0.8998459 ]
  [0.04577699 0.0295856  0.9246374 ]
  ...
  [0.03010805 0.02063832 0.94925356]
  [0.03574431 0.02440508 0.9398507 ]
  [0.05533995 0.03740406 0.907256  ]]

 ...

 [[0.08291093 0.07445236 0.84263676]
  [0.06067361 0.0445674  0.894759  ]
  [0.04861871 0.0308875  0.9204938 ]
  ...
  [0.03010805 0.02063832 0.94925356]
  [0.03574431 0.02440508 0.9398507 ]
  [0.05533994 0.03740405 0.907256  ]]

 [[0.07500049 0.06687

In [ ]:
predictions = model.predict(labels)
print(predictions)

7/7 [==============================] - 9s 471ms/step
[[[0.06333964 0.05903164 0.87762874]
  [0.04261008 0.03624504 0.9211449 ]
  [0.03459289 0.02818183 0.93722534]
  ...
  [0.03016034 0.0207991  0.9490406 ]
  [0.03579594 0.0245733  0.93963075]
  [0.05539455 0.0376142  0.90699124]]

 [[0.06334004 0.05903281 0.8776272 ]
  [0.04261072 0.03624564 0.9211436 ]
  [0.03459318 0.02818213 0.9372247 ]
  ...
  [0.03016033 0.0207991  0.9490406 ]
  [0.03579595 0.02457331 0.93963075]
  [0.05539456 0.0376142  0.90699124]]

 [[0.06333959 0.05903162 0.8776288 ]
  [0.04261006 0.03624502 0.9211449 ]
  [0.03459288 0.02818182 0.93722534]
  ...
  [0.03016034 0.0207991  0.9490406 ]
  [0.03579594 0.0245733  0.93963075]
  [0.05539455 0.0376142  0.90699124]]

 ...

 [[0.06333981 0.05903211 0.8776281 ]
  [0.0426103  0.03624529 0.9211444 ]
  [0.03459303 0.02818192 0.937225  ]
  ...
  [0.03016034 0.0207991  0.9490406 ]
  [0.03579595 0.02457331 0.93963075]
  [0.05539458 0.03761421 0.9069912 ]]

 [[0.06333972 0.05903

#**Generated Catchphrases**

In [ ]:
def remove_duplicates(text):
    words = text.split()
    unique_words = []

    for word in words:
        if word not in unique_words:
            unique_words.append(word)

    return ' '.join(unique_words)

for i, seq in enumerate(labels):
    catchphrases = []
    non_catchphrases = []
    keyword = ""
    not_keyword = ""
    original_text = df['Catchphrases'][i].split()
    predicted_text = df['Text'][i].split()
    for j, label in enumerate(seq):
        if j < len(original_text):  # Check if j is within the bounds of original_text
            if label == 0:  # B
                keyword = original_text[j]
            elif label == 1:  # I
                keyword += " " + original_text[j]
            elif label == 2:  # O
                if not_keyword:
                    non_catchphrases.append(not_keyword)
                    not_keyword = ""
        if label in [0, 1]:
            catchphrases.append(keyword)

    if keyword:
        non_catchphrases.append(keyword)

    catchphrases = remove_duplicates(' '.join(catchphrases))

    print(f"Document {i+1} catchphrases: {catchphrases}")
    #print(f"Document {i+1} non-catchphrases: {', '.join(non_catchphrases)}")

Document 1 catchphrases: proceeding against quashed. ropolitan and 294 the Indian Penal Technology Act, MLegal Keywords: Learned Metropolitan Magistrate Cognizance of offences Sections proceedings Prima facie Section 292 85 Instruments94 67 2000 Criminal Misc. Case 138 Manager Secretary Officer company Consent connivance Neglect be guilty
Document 2 catchphrases: High Court Nos. 12191-12194 of 2016 Leave granted Petitioners-banks Respondent 1 to Personal guarantee Non-performing Recovery Tribunal OA 766 2013 amounts Interlocutory applications passport Arrest warrant Garnishee order Bengaluru Expeditious disposal Ad interim direction Special leave petitions This Indian. The have filed special challenging the dated 04.03.2016 passed by Karnataka at in Writ Petition 2016. had individually advanced loans No. 1. existing 2 and 3 corporate 21.12.2010 a personal respectively, guaranteeing repayment entire amount due. accounts were classified as non-performing assets, 9 before Bengaluru) seeki

#**Evaluation**

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

def flatten_labels(labels):
    return [label for seq in labels for label in seq]

def flatten_true_labels(labels):
    return [label for seq in labels for label in seq if label != 2]

predicted_labels = model.predict(padded_sequences)
predicted_labels = np.argmax(predicted_labels, axis=-1)

flat_predicted_labels = flatten_labels(predicted_labels)
flat_true_labels = flatten_true_labels(labels)

7/7 [==============================] - 4s 540ms/step


In [ ]:
precision = precision_score(labels.flatten(), predicted_labels.flatten(), average='macro')
recall = recall_score(labels.flatten(), predicted_labels.flatten(), average='macro')
f1 = f1_score(labels.flatten(), predicted_labels.flatten(), average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.3195646766169154
Recall: 0.3333333333333333
F1 Score: 0.3263038246369357
